In [1]:
import numpy as np
import pandas as pd
from numba import njit
from my_stuff import MufexKeys
from quantfreedom.enums import *
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.indicators.indicators import *
from quantfreedom.plotting.plot_funcs import plot_candles_1_ind_dif_pane, plot_candles_1_ind_same_pane
from quantfreedom.exchanges.exchange import Exchange
from my_stuff import MufexTestKeys, ApexTestKeys
np.set_printoptions(formatter={"float_kind": "{:0.4f}".format})


mufex_test = Mufex(api_key=MufexTestKeys.api_key, secret_key=MufexTestKeys.secret_key, use_test_net=True,)
mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [2]:
candles = mufex_main.get_candles(symbol="BTCUSDT", timeframe="5m", candles_to_dl=200)
open = candles[:1]
high = candles[:, 2]
low = candles[:, 3]
close = candles[:, 4]
close_shift = np.roll(candles[:, 4],1)

It took 00 mins and 00 seconds to download 200 candles


In [15]:
from quantfreedom.exchanges.apex_exchange.apex import Apex
apex_test = Apex(
    api_key=ApexTestKeys.api_key,
    secret_key=ApexTestKeys.secret_key,
    passphrase=ApexTestKeys.passphrase,
    use_test_net=True
)

In [16]:
apex_test.get_all_order_history()

{'orders': [{'id': '509372710975963500',
   'clientId': '6285562864652465',
   'clientOrderId': '6285562864652465',
   'accountId': '495583522279391852',
   'symbol': 'BTC-USDC',
   'side': 'SELL',
   'price': '28258.0',
   'averagePrice': '',
   'limitFee': '0.000049',
   'fee': '',
   'liquidateFee': '',
   'triggerPrice': '0.0',
   'size': '0.178',
   'type': 'MARKET',
   'createdAt': 1699280717984,
   'updatedTime': 0,
   'expiresAt': 1701874800000,
   'status': 'FILLED',
   'timeInForce': 'IMMEDIATE_OR_CANCEL',
   'reduceOnly': True,
   'isPositionTpsl': False,
   'orderId': '509372710975963500',
   'latestMatchFillPrice': '35078.0',
   'cumMatchFillSize': '0.178',
   'cumMatchFillValue': '6243.8840',
   'cumMatchFillFee': '3.121942',
   'cumSuccessFillSize': '0.178',
   'cumSuccessFillValue': '6243.8840',
   'cumSuccessFillFee': '3.121942',
   'triggerPriceType': 'MARKET',
   'isOpenTpslOrder': False,
   'isSetOpenTp': False,
   'isSetOpenSl': False,
   'openTpParam': {},
   'ope

In [17]:
apex_test.get_trade_history()

{'orders': [{'id': '509372711013712236',
   'clientId': '6285562864652465',
   'clientOrderId': '6285562864652465',
   'accountId': '495583522279391852',
   'symbol': 'BTC-USDC',
   'side': 'SELL',
   'price': '35078.0',
   'averagePrice': '',
   'limitFee': '',
   'fee': '3.121942',
   'liquidateFee': '0.000000',
   'size': '0.178',
   'type': '',
   'direction': 'TAKER',
   'createdAt': 1699280717993,
   'updatedTime': 1699282264776,
   'status': 'SUCCESS_L2_APPROVED',
   'timeInForce': '',
   'matchFillId': '4024a693-feeb-5bc9-be96-fc0f95982559',
   'reduceOnly': False,
   'isPositionTpsl': False,
   'orderId': '509372710975963500',
   'orderType': 'MARKET',
   'exitType': 'Trade',
   'latestMatchFillPrice': '',
   'cumMatchFillSize': '',
   'cumMatchFillValue': '',
   'cumMatchFillFee': '',
   'cumSuccessFillSize': '',
   'cumSuccessFillValue': '',
   'cumSuccessFillFee': '',
   'triggerPriceType': '',
   'isOpenTpslOrder': False,
   'isSetOpenTp': False,
   'isSetOpenSl': False,
 